## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transformaciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [74]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [75]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,estado_credito_ok,falta_pago_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True,True,True,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,0.0,True,True,True,True,True


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila_y,estado_civil_ok,estado_cliente_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok,regla_duracion_credito
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,Blue,111.0,3.0,0.0,True,True,True,True,True,ok


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


-----

Selección de datos (filtrado a nivel filas)

In [76]:
# Se establece qué columnas se eliminan
col_eliminar_creditos = ["completitud_fila", "situacion_vivienda_ok", "objetivo_credito_ok", "estado_credito_ok", "falta_pago_ok", "tasa_interes"]
col_eliminar_tarjetas = ["nivel_tarjeta"]

# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [77]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,0,N


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0,0.0,True,True,True,True,True


-----

Limpieza de los datos (filtrado a nivel filas)

In [78]:
'''
Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente 
mayor al 50% sus ingresos deberán ser mayores a 20.000.
'''

#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,1,Y,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,0,Y,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,0,N,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,0,N,ok


In [79]:
'''
Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos
del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos de los que sea propietario de su vivienda.
'''

#Se puede definir una función para aplicar los cálculos
def regla_duracion_credito(row):
    duracion_credito = row.duracion_credito
    ingresos = row.ingresos
    situacion_vivienda=row.situacion_vivienda
    importe_solicitado=row.importe_solicitado
    pct_ingresos=row.pct_ingreso
    if duracion_credito <= 2:
        if situacion_vivienda != 'PROPIA':
            if pct_ingresos>0.6:
            # Es un error, no cumple la regla definida
                return 'err'
            else:
                return 'ok'
        else:
            #no se aplica la regla para propietarios
            return 'ok'
    else:
        #no se aplica la regla para duraciones mayores a la mínima
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_duracion_credito = df_integrado.apply(lambda row: regla_duracion_credito(row), axis=1).rename("regla_duracion_credito")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_duracion_credito], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos,regla_duracion_credito
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,1,Y,ok,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,0,Y,ok,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,0,N,ok,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,0,N,ok,ok


In [80]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtrado: {df_creditos.shape[0]}")
temp_1 = df_creditos[df_creditos['edad'] < 90]

temp_2 = temp_1[temp_1['antiguedad_empleado'] <= 50] 

temp_3 = temp_2[temp_2['regla_pct_ingresos'] == 'ok']  # Regla 1

temp_4 = temp_3[temp_3['regla_duracion_credito'] == 'ok']  # Regla 2

print(f"Filas después del filtrado: {temp_4.shape[0]}")

Filas antes del filtrado: 10127
Filas después del filtrado: 9765


-----
Integración de datos

In [81]:
df_integrado = pd.merge(temp_4, df_tarjetas, on='id_cliente', how='inner') # Unir los dataset sobre los que hayamos aplicado los filtros
coincidencias = df_integrado.shape[0]

print(f"Cantidad de filas del dataset integrado: {coincidencias}")
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de filas del dataset integrado: 9765
Cantidad de columnas del dataset integrado: 28


-----
Transformación de atributos

* Atributos nominales en los que se modifican los valores:

In [82]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}
estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil')

# Columna: estado_credito
cambios_estado_credito = {
    0 : 'C', # Cancelado
    1 : 'P'  # Pendiente cancelación
}
estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito')

* Atributos numéricos que se discretizan:

In [83]:
df_integrado['operaciones_ult_12m'].describe()

count    9765.00000
mean       64.98382
std        23.52001
min        10.00000
25%        45.00000
50%        67.00000
75%        81.00000
max       139.00000
Name: operaciones_ult_12m, dtype: float64

In [84]:
# antigüedad del empleado

etiquetas_a_e = ['menor_10', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleado_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)
print(antiguedad_empleado_N.value_counts(), "\n")


# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)
print(edad_N.value_counts(), "\n")


# pct ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)
print(pct_ingreso_N.value_counts(), "\n")

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)
print(ingresos_N.value_counts())

antiguedad_empleado
menor_10    4800
5_a_10      4702
mayor_10     263
NA             0
Name: count, dtype: int64 

edad
menor_25    6891
25_a_30     2874
Name: count, dtype: int64 

pct_ingreso
hasta_20    6253
20_a_40     3074
40_a_60      426
mayor_60      12
Name: count, dtype: int64 

ingresos
20k_a_50k     4961
50k_a_100k    3927
mayor_100k     460
hasta_20k      417
Name: count, dtype: int64


In [85]:
# antigüedad del cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)
print(antiguedad_cliente_N.value_counts(), "\n")

# límite crédito tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)
print(limite_credito_tc_N.value_counts(), "\n")


# gastos últimos 12 meses

etiquetas_g_u12 = ['menor_1k', '1k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)
print(gastos_ult_12m_N.value_counts(), "\n")


# operaciones últimos 12 meses

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)
print(operaciones_ult_12m_N.value_counts())

antiguedad_cliente
2y_a_4y     8319
menor_2y     816
mayor_4y     630
Name: count, dtype: int64 

limite_credito_tc
menor_3k     3321
mayor_10k    2657
5k_a_10k     1931
3k_a_5k      1856
Name: count, dtype: int64 

gastos_ult_12m
1k_a_4k      4907
4k_a_6k      3291
mayor_10k     742
6k_a_8k       351
8k_a_10k      320
menor_1k      154
Name: count, dtype: int64 

operaciones_ult_12m
50_a_75      3269
75_a_100     2915
30_a_50      2320
mayor_100     670
15_a_30       567
menor_15       24
Name: count, dtype: int64


In [86]:
df_integrado.columns

Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'antiguedad_empleado', 'situacion_vivienda', 'ingresos',
       'objetivo_credito', 'pct_ingreso', 'estado_credito', 'falta_pago',
       'regla_pct_ingresos', 'regla_duracion_credito', 'antiguedad_cliente',
       'estado_civil', 'estado_cliente', 'gastos_ult_12m', 'genero',
       'limite_credito_tc', 'nivel_educativo', 'operaciones_ult_12m',
       'personas_a_cargo', 'completitud_fila', 'estado_civil_ok',
       'estado_cliente_ok', 'genero_ok', 'nivel_educativo_ok',
       'nivel_tarjeta_ok'],
      dtype='object')

In [87]:
col_eliminar_final = [
              'estado_civil',
              'estado_credito',
              'antiguedad_empleado',
              'edad',
              'pct_ingreso', 
              'ingresos',
              'antiguedad_cliente', 
              'limite_credito_tc', 
              'gastos_ult_12m', 
              'operaciones_ult_12m',
              'regla_pct_ingresos',
              'regla_duracion_credito',
              'id_cliente']

df_integrado.drop(col_eliminar_final, inplace=True, axis=1)

df_final = pd.concat([estado_civil_N, estado_credito_N, antiguedad_empleado_N, edad_N, pct_ingreso_N, ingresos_N, antiguedad_cliente_N, limite_credito_tc_N, gastos_ult_12m_N, operaciones_ult_12m_N, df_integrado], axis=1)

df_final.head(5)

,estado_civil,estado_credito,antiguedad_empleado,edad,pct_ingreso,ingresos,antiguedad_cliente,limite_credito_tc,gastos_ult_12m,operaciones_ult_12m,...,estado_cliente,genero,nivel_educativo,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok
0,C,C,5_a_10,menor_25,hasta_20,50k_a_100k,2y_a_4y,3k_a_5k,mayor_10k,mayor_100,...,ACTIVO,F,SECUNDARIO_COMPLETO,3.0,0.0,True,True,True,True,True
1,S,P,menor_10,menor_25,hasta_20,20k_a_50k,2y_a_4y,3k_a_5k,menor_1k,15_a_30,...,PASIVO,M,DESCONOCIDO,0.0,0.0,True,True,True,True,True
2,D,C,5_a_10,menor_25,20_a_40,20k_a_50k,2y_a_4y,5k_a_10k,1k_a_4k,15_a_30,...,PASIVO,M,POSGRADO_COMPLETO,3.0,0.0,True,True,True,True,True
3,S,C,menor_10,25_a_30,20_a_40,20k_a_50k,menor_2y,menor_3k,1k_a_4k,75_a_100,...,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0,0.0,True,True,True,True,True
4,C,C,menor_10,25_a_30,20_a_40,50k_a_100k,2y_a_4y,3k_a_5k,4k_a_6k,50_a_75,...,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0,0.0,True,True,True,True,True


-----
Exportación de metadatos

In [88]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

-----
Exportación del dataset

In [89]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)